# **Setup**

In [1]:
!pip install diffusers
!pip install transformers
!pip install rembg

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.4 MB/s eta 0:00:00


In [2]:
import torch
from diffusers import DiffusionPipeline, EulerAncestralDiscreteScheduler
import torch
import pandas as pd
import time
import os
import shutil
import torchvision
from PIL import Image
from transformers import pipeline
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
import rembg

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

# **Define Functions**

In [3]:
def sdxl_bg_remove(prompts):
    # Check if cuda is available
    use_cuda = torch.cuda.is_available()
    # Set proper device based on cuda availability
    device = torch.device("cuda" if use_cuda else "cpu")

    #batch_size = 10

    images=[]

    pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
    pipe.to("cuda")
    times = []
    for i, prompt in enumerate(prompts):
        start_time = time.time()
        gen_image = pipe(prompt=prompt).images[0]
        end_time = time.time()
        time_single = end_time - start_time
        times.append(time_single)
        print("SDXL Inference Time: %4f seconds" % (time_single))
        #gen_image.save(f"XL-image{i}.jpg")
        #plt.imshow(gen_image)
        #plt.show()
        images.append(gen_image)

    print(f"SDXL Average Inference Time: {sum(times) / len(times)} seconds")
    print(f"SDXL Total Inference Time: {sum(times)} seconds")
    pipe2 = pipeline("image-segmentation", model="briaai/RMBG-1.4", trust_remote_code=True, device=device)

    final_images = []
    bgrm_times = []
    for i, image in enumerate(images):
        #image_resized = image.resize((376, 263), Image.Resampling.LANCZOS)
        #image_resized.save(f"temp2/XL-image{i}.jpg")
        start_time = time.time()
        pipe2 = pipeline("image-segmentation", model="briaai/RMBG-1.4", trust_remote_code=True, device=device)
        end_time = time.time()
        time_single = end_time - start_time
        bgrm_times.append(time_single)
        print("RMBG Inference Time: %4f seconds" % (time_single))
        #pillow_mask = pipe2(image, return_mask = True) # outputs a pillow mask
        final_image = pipe2(image) # applies mask on input and returns a pillow image
        #final_image.convert('RGB').save(f"XL-image-NoBG{i}.jpg")
        final_images.append(final_image.convert('RGB'))
        #plt.imshow(final_image)
        #plt.show()

    print(f"RMBG Average Inference Time: {sum(bgrm_times) / len(bgrm_times)} seconds")
    print(f"RMBG Total Inference Time: {sum(bgrm_times)} seconds")
    return final_images

In [7]:
def zero123plus(images):
    # Check if cuda is available
    use_cuda = torch.cuda.is_available()
    # Set proper device based on cuda availability
    device = torch.device("cuda" if use_cuda else "cpu")

    times = []
    results = []

    # Load the pipeline
    z123_pipeline = DiffusionPipeline.from_pretrained(
        "sudo-ai/zero123plus-v1.1", custom_pipeline="sudo-ai/zero123plus-pipeline",
        torch_dtype=torch.float16
    )

    # Feel free to tune the scheduler!
    # `timestep_spacing` parameter is not supported in older versions of `diffusers`
    # so there may be performance degradations
    # We recommend using `diffusers==0.20.2`
    z123_pipeline.scheduler = EulerAncestralDiscreteScheduler.from_config(
        z123_pipeline.scheduler.config, timestep_spacing='trailing'
    )
    z123_pipeline.to('cuda:0')

    times = []
    results = []
    for i, image in enumerate(images):
      image = image.resize((360, 360))
      start_time = time.time()
      # Run the pipeline!
      result = z123_pipeline(image, num_inference_steps=100).images[0]
      result = rembg.remove(result)
      results.append(result)
      end_time = time.time()
      time_single = end_time - start_time
      times.append(time_single)
      print(f"Zero123++ Inference time: {time_single} seconds")
    print(f"Zero123++ Average Inference Time: {sum(times) / len(times)} seconds")
    print(f"Zero123++ Total Inference Time: {sum(times)} seconds")
    return results

    # for general real and synthetic images of general objects
    # usually it is enough to have around 28 inference steps
    # for images with delicate details like faces (real or anime)
    # you may need 75-100 steps for the details to constru

In [5]:
def crop_angles(images):
  cropped_images = []
  times = []
  for image in images:
    start_time = time.time()
    image = image.resize((640, 960))
    image1 = image.crop((0, 0, 320, 320))
    image2 = image.crop((320, 0, 640, 320))
    image3 = image.crop((0, 320, 320, 640))
    image4 = image.crop((320, 320, 640, 640))
    image5 = image.crop((0, 640, 320, 960))
    image6 = image.crop((320, 640, 640, 960))
    end_time = time.time()
    cropped_image = [image1, image2, image3, image4, image5, image6]
    cropped_images.append(cropped_image)
    time_single = end_time - start_time
    times.append(time_single)
    print(f"Crop Time: {time_single} seconds")
  print(f"Crop Average Time: {sum(times) / len(times)} seconds")
  print(f"Crop Total Time: {sum(times)} seconds")

  return cropped_images


# **Example From Scratch**

In [20]:
image_folder = "/content/drive/MyDrive/123Plus_images/"
os.makedirs(image_folder, exist_ok=True)
stages_path = ['intermediate/', 'final/', 'cropped/']

# Create Prompts
prompts = [
    'a highland cow',
    "a bulldozer made out of toy bricks",
    "an English castle, aerial view",
    # "a beautifully carved wooden knight chess piece",
    # "a pug made out of metal",
    # "an amigurumi bulldozer",
    # "a corgi puppy",
    # "a crab, low poly",
    # "a bald eagle carved out of wood",
    # "delicious hamburger",
    # "a teal moped",
    # "a fresh cinnamon roll covered in glaze",
    # "a red convertible car with the top down",
    # "a majestic sailboat",
    # "a wedge of cheese on a silver platter",
    # "a silver platter piled high with fruits",
    # "a delicious croissant",
    # "a palm tree, low poly",
    # "a pair of headphones sitting on a desk",
    # "a yellow schoolbus",
    # "an origami motorcycle",
    # "a wooden desk and chair from an elementary school",
    # "an ice cream sundae",
    # "a ladybug",
    # "A DMC Delorean car",
    # "a roast turkey on a platter",
    # "a ripe strawberry",
    # "a blue tulip",
    # "a shiny red stand mixer",
    # "a snail on a leaf"
]

# Folder Creation
for prompt in prompts:
  prompt = prompt.replace(" ", "_")
  prompt = prompt.replace(",", "")
  for stage in stages_path:
    os.makedirs(image_folder + prompt + '/' + stage, exist_ok=True)

# Run SDXL on Prompts
intermediate_images = sdxl_bg_remove(prompts)
for i, image in enumerate(intermediate_images):
    file_name = prompts[i].replace(" ", "_")
    file_name = file_name.replace(",", "")
    image.save(image_folder + file_name + '/' + stages_path[0] + f"intermediate_{file_name}.png")
# Run Zero123++ on SDXL Images
final_images = zero123plus(intermediate_images)
for i, image in enumerate(final_images):
    file_name = prompts[i].replace(" ", "_")
    file_name = file_name.replace(",", "")
    image.save(image_folder + file_name + '/' + stages_path[1] + f"final_{file_name}.png")

# Crop Final Results from Zero123++
cropped_list = crop_angles(final_images)
for i, crop_ims in enumerate(cropped_list):
  file_name = prompts[i].replace(" ", "_")
  file_name = file_name.replace(",", "")
  for idx, image in enumerate(crop_ims):
    image.save(image_folder + file_name + '/' + stages_path[2] + f"cropped_image{idx}.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

SDXL Inference Time: 19.161571 seconds


  0%|          | 0/50 [00:00<?, ?it/s]

SDXL Inference Time: 19.664975 seconds


...

Zero123++ Inference time: 22.045494079589844 seconds


  0%|          | 0/100 [00:00<?, ?it/s]

Zero123++ Inference time: 22.439712047576904 seconds


  0%|          | 0/100 [00:00<?, ?it/s]

Zero123++ Inference time: 22.36456036567688 seconds
Zero123++ Average Inference Time: 22.283255497614544 seconds
Zero123++ Total Inference Time: 66.84976649284363 seconds
Crop Time: 0.0010933876037597656 seconds
Crop Time: 0.001111745834350586 seconds
Crop Time: 0.0008389949798583984 seconds
Crop Average Time: 0.00101470947265625 seconds
Crop Total Time: 0.00304412841796875 seconds


# **Generate From Old Outputs**

In [6]:
def get_folder_names(path):
    """Gets a list of folder names in the specified path."""
    folders = []
    for entry in os.scandir(path):
        if entry.is_dir():
            folders.append(entry.name)
    return folders

In [9]:
parent_dir = "/content/drive/MyDrive/sdxl_test_images/"
stages_path = ['final/', 'cropped/']
prompts = get_folder_names(parent_dir)

for prompt in prompts:
  img_path = parent_dir + prompt + f'/{prompt}_output.png'
  final_path = parent_dir + prompt + '/' + stages_path[0]
  cropped_path = parent_dir + prompt + '/' + stages_path[1]
  os.makedirs(final_path, exist_ok=True)
  image = Image.open(img_path)
  final_image = zero123plus([image])
  final_image[0].save(final_path + f"{prompt}_output.png")
  cropped_list = crop_angles([final_image[0]])
  for idx, crop_ims in enumerate(cropped_list):
    for i, image in enumerate(crop_ims):
      image.save(cropped_path + f"cropped_image{i}.png")

unet/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/8 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--sudo-ai--zero123plus-v1.1/snapshots/36df7de980afd15f80b2e1a4e9a920d7020e2654/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--sudo-ai--zero123plus-v1.1/snapshots/36df7de980afd15f80b2e1a4e9a920d7020e2654/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--sudo-ai--zero123plus-v1.1/snapshots/36df7de980afd15f80b2e1a4e9a920d7020e2654/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--sudo-ai--zero123plus-v1.1/snapshots/36df7de980afd15f80b2e1a4e9a920d7020e2654/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


  0%|          | 0/100 [00:00<?, ?it/s]

Zero123++ Inference time: 21.84354853630066 seconds
Zero123++ Average Inference Time: 21.84354853630066 seconds
Zero123++ Total Inference Time: 21.84354853630066 seconds
Crop Time: 0.0010426044464111328 seconds
Crop Average Time: 0.0010426044464111328 seconds
Crop Total Time: 0.0010426044464111328 seconds


...

unet/diffusion_pytorch_model.safetensors not found


Loading pipeline components...:   0%|          | 0/8 [00:00<?, ?it/s]

An error occurred while trying to fetch /root/.cache/huggingface/hub/models--sudo-ai--zero123plus-v1.1/snapshots/36df7de980afd15f80b2e1a4e9a920d7020e2654/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--sudo-ai--zero123plus-v1.1/snapshots/36df7de980afd15f80b2e1a4e9a920d7020e2654/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /root/.cache/huggingface/hub/models--sudo-ai--zero123plus-v1.1/snapshots/36df7de980afd15f80b2e1a4e9a920d7020e2654/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /root/.cache/huggingface/hub/models--sudo-ai--zero123plus-v1.1/snapshots/36df7de980afd15f80b2e1a4e9a920d7020e2654/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


  0%|          | 0/100 [00:00<?, ?it/s]

Zero123++ Inference time: 22.460063219070435 seconds
Zero123++ Average Inference Time: 22.460063219070435 seconds
Zero123++ Total Inference Time: 22.460063219070435 seconds
Crop Time: 0.0010771751403808594 seconds
Crop Average Time: 0.0010771751403808594 seconds
Crop Total Time: 0.0010771751403808594 seconds
